# Results visualization

In [ ]:
# INPUT 
results_path = '../build/results2.csv'
trial_name = "door1080-h1-3iter-stream" # insert here the trial name

## Just used for Amdahl's law plot
resolution = 720 # insert here the resolution of the video, 1080 or 720

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import functools

# Colorblind friendly colorscheme https://davidmathlogic.com/colorblind/#%23648FFF-%23785EF0-%23DC267F-%23FE6100-%23FFB000
colorscheme = ['#648FFF', '#785EF0', '#DC267F', '#FE6100', '#FFB000'] # px.colors.sequential.Viridis[::3] - import plotly.express as px
symbols = ['circle', 'square', 'x', 'diamond', 'triangle-up']

df = pd.read_csv(results_path)
df.head()

In [ ]:
assert trial_name in df["trial name"].unique(), "Trial name not found in results.csv"

## Execution times
Execution time in microseconds for each single function obtained with the execution of src/meter.cpp

|       	| READ FRAME 	| DEQUE 	| GREY 	| DETECT MOTION 	|
|:-----:	|:----------:	|:-----:	|:----:	|:-------------:	|
| 1080P 	|    9285    	|   1   	| 3758 	|      864      	|
|  720  	|    6010    	|   1   	| 1564 	|      308      	|
            
<br>

|       	| OPENCV_BLUR 	| BOX_BLUR 	|   H1   	| H2/3/4 	|   H1_7  	|
|:-----:	|:-----------:	|:--------:	|:------:	|:------:	|:-------:	|
| 1080P 	|     3846    	|   15128  	| 470467 	| 822913 	| 1520998 	|
|  720  	|     1888    	|   6281   	| 195519 	| 335527 	|  640675 	|

In [ ]:
# Parameters measured with meter.cpp
execution_times = {
    720: {"blur_time" : {"OPEN_CV":1888, "BOX_BLUR": 6281, "H1":195519, "H2-H3-H4":335527, "H1_7":640675}, "serial_time": 6010, "service_time_without_blur": 1564 + 308},
    1080: {"blur_time" : {"OPEN_CV":3846, "BOX_BLUR": 15128, "H1":470467, "H2-H3-H4":822913, "H1_7":1520998}, "serial_time": 9285, "service_time_without_blur": 3758 + 864}
}

df = df[df['trial name'] == trial_name]

In [ ]:
mean = df.groupby(["type", "threads"]).mean().reset_index().sort_values(by=["type", "threads"])
std = df.groupby(["type", "threads"]).std().reset_index().sort_values(by=["type", "threads"])

means = {name:d for name, d in mean.groupby(["type"])}

def compare(x, y):
    if x == "omp": return 1
    elif y == "omp": return -1
    else: return x < y

std = {name:d for name, d in std.groupby(["type"])}
types = sorted(means.keys(), key=functools.cmp_to_key(compare)) # sort by type, but "omp" in the last position (because only sometimes it is used)
threads = pd.unique(mean["threads"]).tolist()

In [ ]:
def plot_amdahl_speedup(width=800, height=400):
    def get_amdahl_speedup(n, serial_fraction, service_time):
        return service_time / (serial_fraction * service_time + (1 - serial_fraction) * service_time / n)

    fig = go.Figure()
    for blur_name, blur_us in execution_times[resolution]["blur_time"].items():
        temp_service_time = execution_times[resolution]["service_time_without_blur"] + blur_us
        serial_fraction = execution_times[resolution]["serial_time"] / temp_service_time
        speedup = [get_amdahl_speedup(thread_number, serial_fraction, temp_service_time) for thread_number in threads]
        limit = temp_service_time / (serial_fraction * temp_service_time)
        fig.add_trace(go.Scatter(x=threads, y=speedup, name=blur_name, mode='lines+markers'))

    for i, trace in enumerate(fig.data):
        # use different symbol and color for each trace
        trace.marker.symbol = symbols[i]
        trace.marker.size = 10
        trace.line.color = colorscheme[i]
        # trace.line.color = "black"

    fig.update_layout(
        width=width,
        height=height,
        title="Amdahl's Speedup",
        xaxis_title="# Threads",
        yaxis_title="Speedup upperbound",
        xaxis = dict(
            tickmode = 'array',
            tickvals = df['threads'].unique(),
        ),
        legend=dict(
            title="Blur algorithm",
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        margin=go.layout.Margin(
            l=0, #left margin
            r=0, #right margin
            b=0, #bottom margin
            t=0  #top margin
        ),
    )
    # LOG 
    fig.update_xaxes(type="log")
    fig.show()

In [ ]:
def plot_metrics(y, title, xtitle, ytitle, log_x=True, log_y=True, width=800, height=400):
    items = means["seq"]["items"]
    fig = go.Figure()

     # Add ideal metric
    if y == "completion time":
        seq_completion_time = means["seq"]["completion time"].iloc[0]
        ideal_completion_time = [seq_completion_time / thread for thread in threads]
        fig.add_trace(go.Scatter(x=threads, y=ideal_completion_time, name="ideal", line=dict(dash="dash")))

        # estimated = [estimated_service_time * items / thread for thread in threads]
        # fig.add_trace(go.Scatter(x=threads, y=estimated, name="lower bound", line=dict(dash="dash")))
    elif y == "service time":
        seq_service_time = means["seq"]["service time"].iloc[0]
        ideal_service_time = [seq_service_time / thread for thread in threads]
        fig.add_trace(go.Scatter(x=threads, y=ideal_service_time, name="ideal", line=dict(dash="dash")))

        # estimated = [estimated_service_time / thread for thread in threads]
        # fig.add_trace(go.Scatter(x=threads, y=estimated, name="lower bound", line=dict(dash="dash")))
    elif y == "efficiency":
        ones = pd.Series(np.ones(len(threads)))
        fig.add_trace(go.Scatter(x=threads, y=ones, name="ideal", line=dict(dash="dash")))
    else:
        fig.add_trace(go.Scatter(x=threads, y=threads, name="ideal", line=dict(dash="dash")))

    # Add the actual metric
    for type in types:
        if type == "seq":
            continue
        fig.add_trace(go.Scatter(x=threads, y=means[type][y], name=type, error_y=dict(type="data", array=std[type][y], visible=True)))

    # Use different symbol and color for each trace
    for i, trace in enumerate(fig.data):
        trace.marker.symbol = symbols[i]
        trace.marker.size = 8
        trace.line.color = colorscheme[i]

    # Update layout
    fig.update_layout(
        width=width,
        height=height,
        title=title,
        xaxis_title=xtitle,
        yaxis_title=ytitle,
        xaxis = dict(
            tickmode = 'array',
            tickvals = threads,
        ),
        legend=dict(
            title="Parallel implementation",
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        margin=go.layout.Margin(
            l=0, #left margin
            r=0, #right margin
            b=0, #bottom margin
            t=0  #top margin
        ),
    )

    # Log scale
    if log_x:
        fig.update_xaxes(type="log")

    if log_y:
        fig.update_yaxes(type="log")

    fig.show()

## Amdahl's speedup

In [ ]:
plot_amdahl_speedup()
# 76, 46, 29, 2.34, 1.62 for 720p
# 100, 66, 43, 3.10, 1.9 for 1080p

## Metrics

In [ ]:
plot_metrics(y="speedup", title="Speedup vs. Number of threads", xtitle="# Threads", ytitle="Speedup")

In [ ]:
plot_metrics(y="service time", title="Service time vs Number of threads", xtitle="# Threads", ytitle="Service Time (µs)")

In [ ]:
plot_metrics(y="completion time", title="", xtitle="# Threads", ytitle="Completion Time (µs)")

In [ ]:
plot_metrics(y="scalability", title="Scalability vs. Number of threads", xtitle="# Threads", ytitle="Scalability")

In [ ]:
plot_metrics(y="efficiency", title="Efficiency vs. Number of threads", xtitle="# Threads", ytitle="Efficiency")